# Numerai Crypto Competitie Voorspellingsmodel met PySpark Preprocessing

Dit notebook implementeert een voorspellingsmodel voor de Numerai/Numerai Crypto competitie met behulp van PySpark voor data preprocessing en XGBoost voor model training.

## Installatie van benodigde packages

In [ ]:
# Mount Google Drive for persistent storage
from google.colab import drive
drive.mount("/content/drive")

# Create directory for Numerai data and models
!mkdir -p "/content/drive/My Drive/Numerai_Crypto"

In [ ]:
# Check if GPU is available
!nvidia-smi

In [ ]:
# Installeer Java (vereist voor Spark)
!apt-get update -qq
!apt-get install -y default-jre > /dev/null
!java -version

# Installeer Spark en PySpark
!pip install -q pyspark==3.1.2

# Installeer XGBoost
!pip install -q xgboost==1.5.0

# Installeer andere benodigde packages
!pip install -q numerapi pandas cloudpickle==2.2.1 pyarrow scikit-learn scipy==1.10.1 matplotlib

## Importeren van benodigde libraries

In [ ]:
from numerapi import NumerAPI
import pandas as pd
import json
import os
import numpy as np
import time
import matplotlib.pyplot as plt

# Spark imports
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DoubleType, ArrayType
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator

# XGBoost imports
import xgboost as xgb
import cloudpickle

## Initialiseren van Spark

In [ ]:
# Initialiseer Spark sessie
spark = SparkSession.builder \
    .appName("NumeraiPySparkPreprocessing") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.cores", "2") \
    .config("spark.driver.extraJavaOptions", "-XX:+UseG1GC") \
    .config("spark.executor.extraJavaOptions", "-XX:+UseG1GC") \
    .getOrCreate()

# Print Spark versie informatie
print(f"Spark version: {spark.version}")

## Initialiseren van de Numerai API

In [ ]:
# Initialiseer de Numerai API client
# Voor het indienen van voorspellingen zijn API keys nodig
# napi = NumerAPI(public_id="UW_PUBLIC_ID", secret_key="UW_SECRET_KEY")
napi = NumerAPI()

## Data downloaden en laden

In [ ]:
# Gebruik een van de nieuwste dataversies
DATA_VERSION = "v5.0"

# Maak een data directory
!mkdir -p {DATA_VERSION}

# Download data
print("Downloading training data...")
napi.download_dataset(f"{DATA_VERSION}/train.parquet")
napi.download_dataset(f"{DATA_VERSION}/features.json")

# Laad feature metadata
feature_metadata = json.load(open(f"{DATA_VERSION}/features.json"))
print("Available feature sets:", list(feature_metadata["feature_sets"].keys()))
features = feature_metadata["feature_sets"]["small"]  # gebruik "small" voor sneller testen, "medium" of "all" voor betere prestaties

## Data laden met PySpark

In [ ]:
# Laad trainingsdata met Spark
print("Loading training data with Spark...")
train_spark = spark.read.parquet(f"{DATA_VERSION}/train.parquet")

# Selecteer alleen de benodigde kolommen
columns_to_select = ["era"] + features + ["target"]
train_spark = train_spark.select(*columns_to_select)

# Downsampling voor snelheid (optioneel)
print("Preparing data for training...")
# Haal unieke era's op en sample 25% (elke 4e era)
unique_eras = [row.era for row in train_spark.select("era").distinct().collect()]
sampled_eras = unique_eras[::4]
train_spark = train_spark.filter(col("era").isin(sampled_eras))

# Bekijk de data
print(f"Training data count: {train_spark.count()}")
print(f"Number of features: {len(features)}")
print(f"Number of eras: {len(sampled_eras)}")

# Toon schema
train_spark.printSchema()

## Data voorbereiden met PySpark ML Pipeline

In [ ]:
# Bereid data voor met Spark ML Pipeline
print("Preparing feature vector with Spark ML Pipeline...")

# Maak een feature vector van alle features
assembler = VectorAssembler(inputCols=features, outputCol="features_vec")

# Standaardiseer de features (optioneel)
scaler = StandardScaler(inputCol="features_vec", outputCol="features_scaled", withStd=True, withMean=True)

# Maak een pipeline
pipeline = Pipeline(stages=[assembler, scaler])

# Fit de pipeline op de trainingsdata
pipeline_model = pipeline.fit(train_spark)

# Transformeer de data
train_prepared = pipeline_model.transform(train_spark)

# Toon een voorbeeld van de getransformeerde data
train_prepared.select("era", "features_scaled", "target").show(5, truncate=True)

## Converteren van Spark DataFrame naar pandas voor XGBoost training

In [ ]:
# Functie om Spark Vector naar numpy array te converteren
@udf(returnType=ArrayType(DoubleType()))
def vector_to_array(vector):
    return vector.toArray().tolist()

# Converteer Spark Vector naar array kolom
train_prepared = train_prepared.withColumn("features_array", vector_to_array("features_scaled"))

# Selecteer alleen de benodigde kolommen voor training
train_for_xgb = train_prepared.select("features_array", "target")

# Converteer Spark DataFrame naar pandas
print("Converting Spark DataFrame to pandas for XGBoost training...")
train_pd = train_for_xgb.toPandas()

# Converteer features_array kolom naar numpy arrays
X_train = np.stack(train_pd["features_array"].values)
y_train = train_pd["target"].values

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")

## Model trainen met XGBoost

In [ ]:
# Train model met XGBoost
print("Training XGBoost model...")
start_time = time.time()

# Converteer data naar DMatrix formaat
dtrain = xgb.DMatrix(X_train, label=y_train)

# Configureer XGBoost parameters
params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'max_depth': 5,
    'eta': 0.01,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'tree_method': 'auto',  # auto selecteert GPU indien beschikbaar
    'seed': 42
}

# Train het model
num_rounds = 2000
xgb_model = xgb.train(params, dtrain, num_rounds)

training_time = time.time() - start_time
print(f"Training completed in {training_time:.2f} seconds")

## Feature importance visualiseren

In [ ]:
# Feature importance visualiseren
feature_importance = xgb_model.get_score(importance_type='gain')
if feature_importance:
    # Sorteer op importance
    sorted_importance = sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)
    features_names = [item[0] for item in sorted_importance[:20]]
    importance_values = [item[1] for item in sorted_importance[:20]]
    
    plt.figure(figsize=(10, 8))
    plt.barh(range(len(features_names)), importance_values)
    plt.yticks(range(len(features_names)), features_names)
    plt.title('XGBoost Feature Importance (top 20)')
    plt.xlabel('Importance (gain)')
    plt.tight_layout()
    plt.show()

## Model opslaan

In [ ]:
# Sla het model op
model_path = "xgb_model.json"
xgb_model.save_model(model_path)
print(f"Model saved as: {model_path}")

## Validatiedata laden en voorbereiden met PySpark

In [ ]:
# Download validatiedata voor testen
print("Downloading validation data for testing...")
napi.download_dataset(f"{DATA_VERSION}/validation.parquet")

# Laad validatiedata met Spark
print("Loading validation data with Spark...")
validation_spark = spark.read.parquet(f"{DATA_VERSION}/validation.parquet")

# Selecteer alleen de benodigde kolommen
columns_to_select = ["era", "data_type"] + features
validation_spark = validation_spark.select(*columns_to_select)

# Filter alleen validatie data
validation_spark = validation_spark.filter(col("data_type") == "validation")

# Neem een kleine subset voor geheugenefficiëntie
validation_spark = validation_spark.limit(1000)

# Transformeer de data met dezelfde pipeline
validation_prepared = pipeline_model.transform(validation_spark)

# Converteer Spark Vector naar array kolom
validation_prepared = validation_prepared.withColumn("features_array", vector_to_array("features_scaled"))

# Selecteer alleen de benodigde kolommen voor voorspelling
validation_for_xgb = validation_prepared.select("features_array")

# Converteer Spark DataFrame naar pandas
validation_pd = validation_for_xgb.toPandas()

# Converteer features_array kolom naar numpy arrays
X_validation = np.stack(validation_pd["features_array"].values)

# Bewaar de originele validatie data voor het maken van de submission
validation_original = validation_spark.toPandas()

## Voorspellingen maken met het model

In [ ]:
# Maak voorspellingen met het model
print("Making predictions...")
dvalidation = xgb.DMatrix(X_validation)
predictions = xgb_model.predict(dvalidation)

# Toon voorspellingen
print("Sample predictions:")
print(predictions[:5])

## Voorspellingsfunctie definiëren

In [ ]:
# Definieer voorspellingsfunctie die werkt met PySpark preprocessing en XGBoost
def predict(
    live_features: pd.DataFrame,
    live_benchmark_models: pd.DataFrame
) -> pd.DataFrame:
    # Converteer pandas DataFrame naar Spark DataFrame
    live_features_spark = spark.createDataFrame(live_features[features])
    
    # Transformeer de data met dezelfde pipeline
    live_features_prepared = pipeline_model.transform(live_features_spark)
    
    # Converteer Spark Vector naar array kolom
    live_features_prepared = live_features_prepared.withColumn("features_array", vector_to_array("features_scaled"))
    
    # Converteer Spark DataFrame naar pandas
    live_features_pd = live_features_prepared.select("features_array").toPandas()
    
    # Converteer features_array kolom naar numpy arrays
    X_live = np.stack(live_features_pd["features_array"].values)
    
    # Maak voorspellingen met het XGBoost model
    dlive = xgb.DMatrix(X_live)
    predictions = xgb_model.predict(dlive)
    
    # Maak submission DataFrame
    submission = pd.Series(predictions, index=live_features.index)
    return submission.to_frame("prediction")

## Voorspellingsfunctie testen

In [ ]:
# Test voorspellingsfunctie
print("Testing prediction function...")
# Maak een lege DataFrame voor benchmark_models (niet gebruikt in onze voorspellingsfunctie)
empty_benchmark = pd.DataFrame(index=validation_original.index)
predictions_df = predict(validation_original, empty_benchmark)

print(f"Predictions shape: {predictions_df.shape}")
print("\nSample predictions:")
print(predictions_df.head())

## Voorspellingsfunctie opslaan met cloudpickle

In [ ]:
# Pickle voorspellingsfunctie
print("Saving prediction function with cloudpickle...")
p = cloudpickle.dumps(predict)
with open("numerai_pyspark_xgb_model.pkl", "wb") as f:
    f.write(p)

print("Prediction function saved as 'numerai_pyspark_xgb_model.pkl'")

## Kaggle specifieke functies voor het opslaan van resultaten

In [ ]:
# Opslaan van resultaten in Kaggle output
# Dit maakt het mogelijk om de resultaten te downloaden of als dataset te gebruiken
try:
    # Maak een output directory
    !mkdir -p /kaggle/working/output
    
    # Kopieer de belangrijke bestanden
    !cp numerai_pyspark_xgb_model.pkl /kaggle/working/output/
    !cp {model_path} /kaggle/working/output/
    
    # Sla de pipeline op
    pipeline_path = "/kaggle/working/output/pipeline_model"
    pipeline_model.save(pipeline_path)
    
    print("Model bestanden opgeslagen in Kaggle output directory")
except Exception as e:
    print(f"Fout bij opslaan in Kaggle output: {e}")

## Voordelen van PySpark Preprocessing

In [ ]:
# Hier zou je een vergelijking kunnen maken tussen standaard preprocessing en PySpark
print("PySpark Preprocessing Voordelen:")
print("1. Gedistribueerde data verwerking voor grote datasets")
print("2. Efficiënte feature engineering met Spark ML Pipeline")
print("3. Betere schaalbaarheid voor complexe transformaties")
print("4. Mogelijkheid om data te verwerken die niet in geheugen past")
print("5. Integratie met verschillende data bronnen en formaten")

## Afsluiten van Spark

In [ ]:
# Sluit Spark sessie af
spark.stop()